In [1]:
import sys
sys.path.append('../')

In [2]:
from win10toast import ToastNotifier
toast = ToastNotifier()

In [3]:
import torch
import itertools
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

from models.train import train, test
from models.models import StateActionModel
from models.utils import set_seed

In [4]:
def pretty(ld, indent=0):
    with open('result.txt', 'w', encoding='utf-8') as file:
        for d in tqdm(ld):
            file.write('{' + '\n')
            for key, value in d.items():
                file.write('\t' * (indent+1) + str(key) + ':' + str(value) + '\n')
                # file.write('\t' * (indent+1) + str(key) + '\n')
                # file.write('\t' * (indent+2) + str(value) + '\n')
            file.write('},\n')

In [5]:
do_train = True

# seed = 4444
# set_seed(seed)

# metric_filter_1 = 'test_mcc'
# metric_filter_2 = 'val_mcc'

data_dir = '../yarnScripts'
log_path = './logs'
save_path = './saved'

## BERT Fine-tuned

In [6]:
save_model = f'{save_path}_fnn'
log_dir = f'{log_path}_fnn'

dict_model = dict(
    shared_out_dim=[125],
    state_layers=[[20], [30]],
    action_layers=[[20], [30]],
    out_features=[1],
    lstm_model=[False],
    bert_name=["bert-base-multilingual-cased"],
)

list_model = [dict(zip(dict_model.keys(), k)) for k in itertools.product(*dict_model.values())]

In [8]:
if do_train:
    for d in tqdm(list_model):
        d = d.copy()
        # lr = d.pop('lr')

        train(
            model=StateActionModel(**d),
            dict_model=d,
            log_dir=log_dir,
            data_path=data_dir,
            save_path=save_model,
            lr=1e-3,
            optimizer_name="adamw",
            n_epochs=3000,
            batch_size=8,
            num_workers=0,
            scheduler_mode='max_val_acc',
            debug_mode=False,
            steps_validate=1,
            use_cpu=False,
            freeze_bert=True,
            balanced_actions=True,
        )

  0%|          | 0/4 [00:00<?, ?it/s]

#### Results

In [ ]:
test(
    data_path=data_dir,
    save_path=save_model,
    n_runs=1,
    batch_size=8,
    num_workers=0,
    debug_mode=False,
    use_cpu=False,
    save=True,
)

toast.show_toast("Done")

In [ ]:
res_edges = res_edges_sage
res_edges[0]

In [ ]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_1] for k in all])[::-1]
all[sort_idx[0]]['dict']

In [ ]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_2] for k in all])[::-1]
all[sort_idx[0]]['dict']

In [ ]:
pretty([all[k]['dict'] for k in sort_idx])

#### cm

In [ ]:
cm = all[sort_idx[0]]['test_cm'][0]

In [ ]:
cm.visualize()

In [ ]:
cm.class_accuracy

In [ ]:
cm.pseudo_perc.max()

In [ ]:
[mean_squared_error(y_true=cm.true_percentiles[cm.labels==k], y_pred=cm.pseudo_perc[cm.labels==k], squared=False) for k in range(0, cm.size)]